<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random

In [ ]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

[]

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

# path to the folder where the dataset is located
path = '/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0'

# assigning training data path to the target language
# and reading the tsv file
train_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.train.tsv")
train=pd.read_csv(train_path,sep="\t",names=['tgt','src','count'])

# assigning validation data path to the target language
# and reading the tsv file 
valid_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.dev.tsv")
valid=pd.read_csv(valid_path,sep="\t",names=['tgt','src','count'])

# assigning test data path to the target language
# and reading the tsv file
test_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.test.tsv")
test=pd.read_csv(test_path,sep="\t",names=['tgt','src','count'])




In [ ]:
# Function to assign numbers to characters and characters to numbers
# for the vocabulary passed to it, be it source language vocab or target
# language vocab

def assign(vocab):
  char_int=dict([(char, i) for i, char in enumerate(vocab)])
  int_char=dict((i, char) for char, i in char_int.items())
  return char_int,int_char

In [ ]:
def one_hot_encoding(src,tgt,s_char,t_char,source_char_to_int=None,target_char_to_int=None):
  encoder_tokens=len(s_char)
  decoder_tokens=len(t_char)
  max_s_len=max([len(txt) for txt in src])
  max_t_len=max([len(txt) for txt in tgt])
  
  src_vocab=None
  tgt_vocab=None
  
  if source_char_to_int==None and target_char_to_int== None:
    # Assigning numbers to the characters for the source language 
    source_char_int,source_int_char=assign(s_char)
    src_vocab=(source_char_int,source_int_char)

    # Assigning characters to number for the target language
    target_char_int,target_int_char= assign(t_char)
    tgt_vocab=(target_char_int,target_int_char)

  # Input to encoder and decoder
  encoder_input=np.zeros((len(src),max_s_len,encoder_tokens),dtype="float32")
  decoder_input=np.zeros((len(src),max_t_len,decoder_tokens),dtype="float32")
  
  # Decoder target
  decoder_target=np.zeros((len(src),max_t_len,decoder_tokens),dtype="float32")

 

  for i,(input_text,target_text) in enumerate(zip(src,tgt)):
    
    # Creating one hot encoding for each character in source language
    for t,char in enumerate(input_text):
      encoder_input[i, t, source_char_int[char]]=1.0
    encoder_input[i, t + 1 :,source_char_int[" "]]=1.0
    
    # Creating one hot encoding for each character in target language
    for t,char in enumerate(target_text):
      decoder_input[ i, t , target_char_int[char]]=1.0
      if t>0:
        decoder_target[i,t-1,target_char_int[char]]=1.0
      decoder_input[i,t+1:,target_char_int[" "]]=1.0
      decoder_target[i,t:,target_char_int[" "]]=1.0
  
  if src_vocab!=None and tgt_vocab!=None:
    return encoder_input,decoder_input,decoder_target,src_vocab,tgt_vocab
  
  else:
    return encoder_input,decoder_input,decoder_target
  

In [ ]:
def data_preprocessing(source,target):
  s_char=set()
  t_char=set()

  source=[str(x) for x in source]
  target=[str(x) for x in target]
  
  s_words=[]
  t_words=[]

  for src,tgt in zip(source,target):
    tgt="\t"+tgt+"\n"
    s_words.append(src)
    t_words.append(tgt)

    # extracting each character from the words and adding them to the respective set
    for char in src:
      if char not in s_char:
        s_char.add(char)
    for char in tgt:
      if char not in t_char:
        t_char.add(char)

  # sorting the characters of source and target languages
  s_char=sorted(list(s_char))
  t_char=sorted(list(t_char))

  # Adding a space
  s_char.append(" ")
  t_char.append(" ")

  return one_hot_encoding(s_words,t_words,s_char,t_char)


In [ ]:
train_source=train["src"].to_list()
train_target=train["tgt"].to_list()

valid_source=valid["src"].to_list()
valid_target=valid["tgt"].to_list()

test_source=test["src"].to_list()
test_target=test["tgt"].to_list()

# Training data
encoder_input_train,decoder_input_train,decoder_target_train,train_src_vocab,train_tgt_vocab = data_preprocessing(train_source,train_target)

# Validation data
encoder_input_valid,decoder_input_valid,decoder_target_valid,valid_src_vocab,valid_tgt_vocab = data_preprocessing(valid_source,valid_target)

# Test data
encoder_input_test,decoder_input_test,decoder_target_test, test_src_vocab,test_tgt_vocab = data_preprocessing(test_source,test_target)


In [ ]:
source_char2int , source_int2char = train_src_vocab
target_char2int , target_int2char =  train_tgt_vocab

In [ ]:
def rnn(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint,hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  e_in=Input(shape=(None,len(src_charint)))
  e_out=e_in

  for i in range(1,num_encoders+1):
    encoder=SimpleRNN(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    e_out,state=encoder(e_in)
  e_states=[state]

  d_in=Input(shape=(None,len(tgt_charint)))
  d_out=d_in

  for i in range(1,num_decoders+1):
    decoder=SimpleRNN(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    d_out,_=decoder(d_in,initial_state=e_states)
  
  hidden=Dense(hidden_layer_size,activation="relu")
  h_out=hidden(d_out)
  d_dense=Dense(len(tgt_charint),activation="softmax")
  d_out=d_dense(h_out)
  model=Model([e_in,d_in],d_out)
  return model


In [ ]:
def lstm(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint,hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  e_in=Input(shape=(None,len(src_charint)))
  e_out=e_in

  for i in range(1,num_encoders+1):
    encoder=LSTM(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    e_out,state=encoder(e_in)
  e_states=[state]

  d_in=Input(shape=(None,len(tgt_charint)))
  d_out=d_in

  for i in range(1,num_decoders+1):
    decoder=LSTM(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    d_out,_=decoder(d_in,initial_state=e_states)
  
  hidden=Dense(hidden_layer_size,activation="relu")
  h_out=hidden(d_out)
  d_dense=Dense(len(tgt_charint),activation="softmax")
  d_out=d_dense(h_out)
  model=Model([e_in,d_in],d_out)
  return model

In [ ]:
def gru(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint,hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  e_in=Input(shape=(None,len(src_charint)))
  e_out=e_in

  for i in range(1,num_encoders+1):
    encoder=GRU(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    e_out,state=encoder(e_in)
  e_states=[state]

  d_in=Input(shape=(None,len(tgt_charint)))
  d_out=d_in

  for i in range(1,num_decoders+1):
    decoder=GRU(embed_size,return_state=True,return_sequences=True,dropout=dropout)
    d_out,_=decoder(d_in,initial_state=e_states)
  
  hidden=Dense(hidden_layer_size,activation="relu")
  h_out=hidden(d_out)
  d_dense=Dense(len(tgt_charint),activation="softmax")
  d_out=d_dense(h_out)
  model=Model([e_in,d_in],d_out)
  return model

In [ ]:
def transliteration(config,src_charint,tgt_charint):
  num_encoders=config["num_encoders"]
  cell=config["cell"]
  embed_size=config["embed_size"]
  dropout=config["dropout"]
  num_decoders=config["num_decoders"]
  hidden_layer_size=config["hidden_layer_size"]

  if cell == "RNN":
    model=rnn(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint,hidden_layer_size)
    return model
  elif cell == "LSTM":
    model=lstm(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint)
    return model
  elif cell == "GRU":
    model=gru(src_charint,num_encoders,embed_size,dropout,num_decoders,tgt_charint)
    return model
  
